In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("../input/titanic/train.csv")
submission = train_data["PassengerId"]
#687 out of 891 rows are nan in cabin so I will drop it too
train_data = train_data.drop(["Name","Ticket","PassengerId","Cabin"],axis=1)
y = train_data["Survived"]
X = train_data.drop("Survived",axis=1)

nan_embarked = X["Embarked"][X["Embarked"].isna()].index
#Deleting 2 rows that has NaN in embarked column
X = X.drop(nan_embarked)
y = y.drop(nan_embarked)
y.reset_index(drop=True)
X.reset_index(drop=True)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
884,2,male,27.0,0,0,13.0000,S
885,1,female,19.0,0,0,30.0000,S
886,3,female,NaN,1,2,23.4500,S
887,1,male,26.0,0,0,30.0000,C


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
from sklearn.preprocessing import OneHotEncoder
X_train["Sex"] = [1 if x=="female" else 0 for x in X_train["Sex"]]
X_valid["Sex"] = [1 if x=="female" else 0 for x in X_valid["Sex"]]
# Apply one-hot encoder to each column with categorical dataonehotencoder1 = OneHotEncoder(categorical_features = [0])
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(pd.DataFrame(X_train["Embarked"])))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(pd.DataFrame(X_valid["Embarked"])))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop("Embarked", axis=1)
num_X_valid = X_valid.drop("Embarked", axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

X_train = imputed_X_train
X_valid = imputed_X_valid

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [5]:
X_train.columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', "0", "1", "2"]
X_valid.columns = X_train.columns

In [6]:
#Training the model
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(max_iter = 1000)
log.fit(X_train,y_train)
predictions = log.predict(X_valid)

In [7]:
log.score(X_valid,y_valid)

0.7902621722846442

In [8]:
deneme = pd.read_csv("../input/titanic/gender_submission.csv")

In [9]:
test_data = pd.read_csv("../input/titanic/test.csv")


In [10]:
#Preprocessing the test data
test_data_index = test_data["PassengerId"]
test_data = test_data.drop(["Name","Ticket","PassengerId","Cabin"],axis=1)
test_data["Sex"] = [1 if x=="female" else 0 for x in test_data["Sex"]]
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_test_data = pd.DataFrame(OH_encoder.fit_transform(pd.DataFrame(test_data["Embarked"])))
OH_cols_test_data.index = test_data.index
num_test_data =test_data.drop("Embarked", axis=1)
test_data = pd.concat([num_test_data, OH_cols_test_data], axis=1)
test_data.columns = X_train.columns
my_imputer = SimpleImputer()
imputed_test_data = pd.DataFrame(my_imputer.fit_transform(test_data))
imputed_test_data.columns =test_data.columns


In [11]:
final_predictions = pd.DataFrame(np.array([test_data_index,log.predict(imputed_test_data)]).T)
final_predictions.columns = ["PassengerId","Survived"]
final_predictions.to_csv('submission.csv', index=False)